**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

**To do:**
- Add dropout
- Implement another loss function

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
#name = 'bpi_2012'
name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process' 

parser = {
    'train': True,
    'test': True,
    'model_class': 'VAE',
    'model_name': ' ',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.3,
    'input_dir': '../input/{}/'.format(name), 
    'output_dir': './output/{}/'.format(name),
    'batch_size' : 16,
    'epochs' : 100,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 1000,
    'layer2': 100,
    'early_stopping': 30,
    'lr': 0.002,
    'betas': (0.9, 0.99),   
    'lr_decay': 0.90,
}

args = argparse.Namespace(**parser)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

## Validate and test

In [12]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [13]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [14]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [15]:
missing_true_val.shape, missing_true_test.shape

((1096, 5), (1071, 5))

In [16]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [17]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [18]:
pd.isnull(normalized_missing_df).sum()

CaseID                  0
Activity             2022
CompleteTimestamp    1974
CumTimeInterval      1974
NormalizedTime       1974
dtype: int64

In [19]:
pd.isnull(missing_true_val).sum()

CaseID                 0
Activity             342
CompleteTimestamp    309
CumTimeInterval      309
NormalizedTime       309
dtype: int64

In [20]:
pd.isnull(missing_true_test).sum()

CaseID                 0
Activity             325
CompleteTimestamp    307
CumTimeInterval      307
NormalizedTime       307
dtype: int64

# Build model

## Define model

In [21]:
if args.model_class == 'VAE':
    model = VAE(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2, args.cuda)

if args.cuda:
    model.cuda()

## Define loss

In [22]:
# Define loss

def loss_function(recon_x, x, mu, logvar, avai_mask): 
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False)
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    
    loss = BCE+KLD
    return loss

In [23]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [24]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
lambda1 = lambda epoch: args.lr_decay ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [25]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [26]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [27]:
def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, nan_matrix_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    nan_matrix_val = Variable(torch.Tensor(nan_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        nan_matrix_val = nan_matrix_val.cuda()
        
    recon_data, mu, logvar = model(m_val)
        
    loss = loss_function(recon_data, c_val, mu, logvar, nan_matrix_val)
    return loss.data[0]/missing_matrix_w_normalized_time_val.shape[0]

```python
def val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
       pad_matrix_val, cols_w_normalized_time, val_row_num,
       nan_time_index_val, nan_activity_index_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        
    recon_val, mu, logvar = model(m_val)
    
    recon_df_w_normalized_time = convert2df(recon_val, pad_matrix_val, cols_w_normalized_time, val_row_num)
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_val, min_max_storage)
    submission_df = getSubmission(recon_df_w_time, missing_true_val, complete_true_val, first_timestamp)
    
    #evaluate
    mae_time, rmse_time, acc = evaluation(submission_df, nan_time_index_val, nan_activity_index_val)
    
    
    return mae_time/86400+1/acc
 ```

In [28]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,893,NaN,2012-01-14 04:03:40,189490438.0,0.000000
1,893,Accepted-In Progress,2012-01-31 22:16:55,191024833.0,0.507308
2,893,Queued-Awaiting Assignment,NaN,NaN,NaN
3,893,Accepted-In Progress,2012-01-31 23:02:59,191027597.0,0.508222
4,893,Completed-Closed,2012-02-18 04:13:20,192515018.0,1.000000


In [29]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,893,Queued-Awaiting Assignment,2012-01-14 04:03:40,189490438.0,0.000000
1,893,Accepted-In Progress,2012-01-31 22:16:55,191024833.0,0.507308
2,893,Queued-Awaiting Assignment,2012-01-31 22:29:08,191025566.0,0.507551
3,893,Accepted-In Progress,2012-01-31 23:02:59,191027597.0,0.508222
4,893,Completed-Closed,2012-02-18 04:13:20,192515018.0,1.000000


# Train model

In [30]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader,
                                                     complete_matrix_w_normalized_time_trainLoader,
                                                     avai_matrix_trainLoader)):
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())
      
        #Transform: np --> Tensor/Variable: tensor --> tensor with wrapper
        #Wraps a tensor and records the operations applied to it.
        #Variable is a thin wrapper around a Tensor object, that also holds the gradient
        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()
            
        optimizer.zero_grad()
        
        recon_data, mu, logvar = model(m_data)
        
        loss = loss_function(recon_data, c_data, mu, logvar, avai_mask)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [32]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        scheduler.step()
        
        train_loss = train(epoch, model, optimizer)
        #val_score = val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
        #                    pad_matrix_val, cols_w_normalized_time, val_row_num, 
        #                    nan_time_index_val, nan_activity_index_val)
        val_score = val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val)
        
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
        
        #method 2 scheduler
        #scheduler.step(val_score)
        
        end = time.time()
        print('====> Epoch {} | Time: {} ms| Train loss: {:.4f} | Val score: {:.4f}'.format(
                          epoch, (end-init)*1000 ,train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Time: 1.1452720165252686 | Train loss: 6.1345 | Val score: 4.3891
====> Epoch 2 | Time: 1.1020219326019287 | Train loss: 5.8269 | Val score: 4.4738
====> Epoch 3 | Time: 1.2038190364837646 | Train loss: 6.0327 | Val score: 4.3081
====> Epoch 4 | Time: 1.2976469993591309 | Train loss: 5.9991 | Val score: 4.1811
====> Epoch 5 | Time: 1.2902140617370605 | Train loss: 5.9853 | Val score: 4.3626
====> Epoch 6 | Time: 1.3427588939666748 | Train loss: 6.0183 | Val score: 4.4264
====> Epoch 7 | Time: 1.437209129333496 | Train loss: 5.8073 | Val score: 4.3683
====> Epoch 8 | Time: 1.6142430305480957 | Train loss: 6.0344 | Val score: 4.6294
====> Epoch 9 | Time: 1.7477359771728516 | Train loss: 5.9703 | Val score: 4.2319
====> Epoch 10 | Time: 1.8503599166870117 | Train loss: 5.8637 | Val score: 4.3557
====> Epoch 11 | Time: 1.870811939239502 | Train loss: 5.9529 | Val score: 4.3940
====> Epoch 12 | Time: 1.582139015197754 | Train loss: 5.8654 | Val score: 4.1992
====> Epoch 13 |

Process Process-265:
Process Process-266:
Process Process-267:
Process Process-270:
Process Process-268:
Process Process-269:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _

# Predict and evaluate

In [32]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test, mu, logvar = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
        
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    
    print('\n')
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...


/home/pllab/miniconda2/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 14501
Mean Absolute Error: 0.9803 day(s)
Root Mean Squared Error: 2.1182 day(s)
Number of missing Activity: 14349
Accuracy: 69.05%


Saving submission...
Done!


In [33]:
submission_df.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,10470,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.047,2012-02-04 01:17:11.047
1,10470,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.323,2012-02-04 01:17:11.097427
2,10470,A_DECLINED-COMPLETE,A_DECLINED-COMPLETE,1.086354e+07,1.086354e+07,2012-02-04 01:17:42.964,2012-02-04 01:17:42.964
3,10471,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:41.949,2012-02-04 01:23:41.949
4,10471,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:42.504,2012-02-04 01:23:42.504
5,10471,A_PREACCEPTED-COMPLETE,A_PREACCEPTED-COMPLETE,1.086394e+07,1.086394e+07,2012-02-04 01:24:23.379,2012-02-04 01:24:23.379
6,10471,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,1.086394e+07,1.086394e+07,2012-02-04 01:24:24.052,2012-02-04 01:24:24.052
7,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087644e+07,1.087644e+07,2012-02-04 04:52:43.090,2012-02-04 04:52:43.090
8,10471,W_Completeren aanvraag-COMPLETE,W_Completeren aanvraag-COMPLETE,1.087696e+07,1.087696e+07,2012-02-04 05:01:19.935,2012-02-04 05:01:19.935
9,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087697e+07,1.087697e+07,2012-02-04 05:01:31.267,2012-02-04 05:01:31.267


In [34]:
submission.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,10470,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.047,2012-02-04 01:17:11.047
1,10470,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086351e+07,1.086351e+07,2012-02-04 01:17:11.323,2012-02-04 01:17:11.097427
2,10470,A_DECLINED-COMPLETE,A_DECLINED-COMPLETE,1.086354e+07,1.086354e+07,2012-02-04 01:17:42.964,2012-02-04 01:17:42.964
3,10471,A_SUBMITTED-COMPLETE,A_SUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:41.949,2012-02-04 01:23:41.949
4,10471,A_PARTLYSUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,1.086390e+07,1.086390e+07,2012-02-04 01:23:42.504,2012-02-04 01:23:42.504
5,10471,A_PREACCEPTED-COMPLETE,A_PREACCEPTED-COMPLETE,1.086394e+07,1.086394e+07,2012-02-04 01:24:23.379,2012-02-04 01:24:23.379
6,10471,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-SCHEDULE,1.086394e+07,1.086394e+07,2012-02-04 01:24:24.052,2012-02-04 01:24:24.052
7,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087644e+07,1.087644e+07,2012-02-04 04:52:43.090,2012-02-04 04:52:43.090
8,10471,W_Completeren aanvraag-COMPLETE,W_Completeren aanvraag-COMPLETE,1.087696e+07,1.087696e+07,2012-02-04 05:01:19.935,2012-02-04 05:01:19.935
9,10471,W_Completeren aanvraag-START,W_Completeren aanvraag-START,1.087697e+07,1.087697e+07,2012-02-04 05:01:31.267,2012-02-04 05:01:31.267


In [35]:
#bpi2013

#30%
#8.1
#75%

#40%
#9.7
#70%

#50%
#10.7
#69%

In [36]:
#bpi2012
#0.59
#75%

In [37]:
#helpdesk
#3.9
#85